# Brain Tumor Auto-Segmentation for MRI

## Description

Magnetic Resonance Imaging (MRI) is an imaging technique which is used to obeserve a variety of diseases and body parts. The atoms of body parts are subjected to magnetic field to which they emmit radio waves. These radio waves are then measured to get an image of the particular body part, this is how [MRI](https://en.wikipedia.org/wiki/Magnetic_resonance_imaging) works

In [1]:
# Just colab things
onColab = True
if onColab:
  !rm -rf repo
  !git clone https://github.com/RohitLad/BrainTumorMRI.git repo
  repoDir = 'repo/'
  %tensorflow_version 1.15.0
else:
  repoDir = ''

Cloning into 'repo'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 70 (delta 10), reused 60 (delta 4), pack-reused 0
Unpacking objects: 100% (70/70), done.
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.0`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


## 1. Import Packages andf Functions

We would be using the following packages

- `numpy` and `pandas` for data manipulation
- `keras` for building Deep Learning models
- `matplotlib` and `seaborn` for plots
- `nibabel` to extract images and labels from dataset

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nibabel as nib
import h5py
from functools import partial

import keras
from keras import backend as K
from keras.layers import Activation, Conv3D, Deconvolution3D, MaxPooling3D, UpSampling3D
from keras.engine import Input, Model
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
K.set_image_data_format("channels_first")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


## 2. Dataset Description

MR images are often encountered in [DICOM format](https://en.wikipedia.org/wiki/DICOM), these images can be processed by using the [pydicom](https://pydicom.github.io/pydicom/stable/getting_started.html) library.

In this project, we shall be using data from the [Decathlon 10 Challenge](https://decathlon-10.grand-challenge.org). The data is mostly preprocessed but in real prectise, MRI data has to be significantly preprocessed in order to use it to train our models.

The dataset is stored in [NifTI-1 format](https://nifti.nimh.nih.gov/nifti-1/) and we will be using the [NiBabel library](https://github.com/nipy/nibabel) to interact with the files. Every training sample consists of two files:

The first file is a 3D image with 4 channels, so basically it is a 4D array of shape (240, 240, 155, 4)

- The first three dimensions signify the location (X, Y and Z coordinates) for each point in the 3D volume
- The 4th dimension is the values of four different sequences namely:
    - FLAIR: "Fluid Attenuated Inversion Recovery"
    - T1w: "T1-weighted"
    - t1gd: "T1-weighted with gadolinium contrast enhancement"
    - T2w: "T2-weighted"
    
The second file is a label file containing of a 3D array of shape (240, 240, 155). The labels are integers corresponding:
   - 0: background
   - 1: edema
   - 2: non-enhancing tumor
   - 3: enhancing tumor

We have 484 training examples which are then split into 80% training and 20% validation data.

In [0]:
# specifying the paths
DataDir = repoDir+"Data/"
trainDir = DataDir+'train/'
validDir = DataDir+'valid/'

## 3. Dice Similarity Coefficient

A natural candidate that one comes up with as a loss function is the cross-entropy loss function. But this function is not suitable for segmentation tasks since there is a heavy class imbalance as there aren't many positive regions in an MR image dataset.

A much more common loss for segmentation tasks is the Dice similarity coefficient (DSC). A DSC measures how similar two regions are or in general, how well two contours overlap.

- Dice index ranges from 0 (complete mismatch) to 1 (perfect match)

For two sets $A$ and $B$, the DSC is defined as:
$$\text{DSC}(A, B) = 2 \times \frac{|A \cap B|}{|A| + |B|}$$
Here, we can interpret $A$ as the prediction for voxel being considered and $B$ being the ground truth.

Considering:
- $x$: the input image
- $f(x)$: the model output (prediction)
- $y$ the label (ground truth)

$$\text{DSC}(f, x, y) = 2 \times\frac{ \sum_{i, j} f(x)_{ij} \times y_{ij} + \epsilon}{\sum_{i,j} f(x)_{ij} + \sum_{i, j} y_{ij} + \epsilon}$$

- $\epsilon$ is a small number to avoid division by zero

<img src="https://www.researchgate.net/publication/328671987/figure/fig4/AS:688210103529478@1541093483784/Calculation-of-the-Dice-similarity-coefficient-The-deformed-contour-of-the-liver-from.ppm" width="30%">

[Image Source](https://www.researchgate.net/figure/Calculation-of-the-Dice-similarity-coefficient-The-deformed-contour-of-the-liver-from_fig4_328671987)


The above is a formulation for a single class. Given $\text{DSC}(f, x, y)$ for a single class, we can think about a multiple class approach.

Consider $\text{DSC}_i(f, x, y)$ be the Dice Coefficient for $i^{th}$ class, we can take an average over $N$ classes and therefore say

$$DC(f, x, y) = \frac{1}{N} \sum_{i=1}^{L} \left ( DSC_{i}(f, x, y) \right )$$

Since we want segmentations for each of three classes of conditions: edema, enhancing tumor, non-enhancing tumor, $L$ would be 3.

## 4. Soft Dice Loss

Since the Dice Similarity Coefficient takes in discrete values, we need an analogous formulation which takes in real valued input. This is where *Soft Dice Loss* comes in:

Considering: 
- $p$: the predictions
- $q$: the ground truth (wither 0 or 1) 
- $\epsilon$ is a small number to avoid division by zero

the *Soft Dice Loss* ${L}_{Dice}$) is given by
$$\mathcal{L}_{Dice}(p, q) = 1 - 2\times\frac{\sum_{i, j} p_{ij}q_{ij} + \epsilon}{\left(\sum_{i, j} p_{ij}^2 \right) + \left(\sum_{i, j} q_{ij}^2 \right) + \epsilon}$$

and as it is understood, for multiple classes

$$\mathcal{L}_{Dice}(p, q) = 1 - \frac{1}{N} \sum_{c=1}^{L} 2\times\frac{\sum_{i, j} p_{cij}q_{cij} + \epsilon}{\left(\sum_{i, j} p_{cij}^2 \right) + \left(\sum_{i, j} q_{cij}^2 \right) + \epsilon}$$


In [0]:
def diceCoefficient(yTrue, yPred, axis=(1,2,3), eps=1e-5):
    """
    Calculate the dice coefficient over all classes
    
    yTrue: tensor of ground truth values (numClasses, yDim, yDim, zDim)
    yPred: tensor of soft predictions (numClasses, yDim, yDim, zDim)
    axis: spatial axes to sum over while computing Numerator and Denominator
    eps: small constant to avoid dividing by zero
    
    returns:
    diceCoefficient: computed value of soft dice coefficient
    """
    numerator = 2*K.sum(yTrue*yPred, axis=axis)+eps
    denominator = K.sum(yPred, axis=axis)+K.sum(yTrue, axis=axis)+eps
    return K.mean(numerator/denominator)

def softDiceLoss(yTrue, yPred, axis=(1,2,3), eps=1e-5):
    """
    Calculate the soft dice loss over all classes
    
    yTrue: tensor of ground truth values (numClasses, yDim, yDim, zDim)
    yPred: tensor of soft predictions (numClasses, yDim, yDim, zDim)
    axis: spatial axes to sum over while computing Numerator and Denominator
    eps: small constant to avoid dividing by zero
    
    returns:
    diceLoss: computed value of soft dice loss
    """
    
    numerator = 2*K.sum(yTrue*yPred, axis=axis)+eps
    denominator = K.sum(yPred**2, axis=axis)+K.sum(yTrue**2, axis=axis)+eps
    return 1. - K.mean(numerator/denominator)

## 5. Model: 3D U-Net

A [3D U-net](https://arxiv.org/abs/1606.06650) is used for this task which takes advantage of the volumetric shape of MR images and is one of the best performing models for this task. Know more about the architecture from [this paper](https://arxiv.org/abs/1606.06650).

<img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" width="50%">


Let us create the model:

In [0]:
class UNet3D:
    
    def __init__(self, optimizer, lossFunction, inputShape=(4,160,160,16), poolSize=(2,2,2), 
                 nLabels=3, lr=1e-5, depth=4, nBaseFilters=32, activation='sigmoid', metrics=[], batchNormalization=False, deconvolution=False):
        self.optimizer = optimizer
        self.lossFunction = lossFunction
        self.inputShape = inputShape
        self.poolSize = poolSize
        self.nLabels = nLabels
        self.learningRate = lr
        self.depth = depth
        self.nBaseFilters = nBaseFilters
        self.activation = activation
        self.metrics = [metrics] if not isinstance(metrics, list) else metrics
        self.deconvolution = deconvolution
        self.batchNormalization = batchNormalization
        self.levels = []
        self.model = None
    
    def _createConvBlock(self, inputLayer, nFilters, batchNorm=False, instanceNorm=False, kernel=(3,3,3), activation='relu', padding='same', strides=(1,1,1)):
        layer = Conv3D(nFilters, kernel, padding=padding, strides=strides)(inputLayer)
        return Activation(activation)(layer)
    
    def _createUpConvBlock(self, nFilters, poolSize, kernelSize=(2,2,2), strides=(2,2,2), deconvolution=False):
        if deconvolution:
            return Deconvolution3D(filters=nFilters, kernel_size=kernelSize, strides=strides)
        else:
            return UpSampling3D(size=poolSize)
    
    def _getCurrFilters(self, depth):
        return self.nBaseFilters*(2**depth)
    
    def createModel(self):
        inputs = Input(self.inputShape)
        currLayer = inputs
        self.levels = []
        self.model = None
        
        for d in range(self.depth):
            currFilters = self._getCurrFilters(depth=d)
            layer1 = self._createConvBlock(inputLayer=currLayer, nFilters=currFilters, batchNorm=self.batchNormalization)
            layer2 = self._createConvBlock(inputLayer=layer1, nFilters=currFilters*2, batchNorm=self.batchNormalization)
            if d < self.depth-1:
                currLayer = MaxPooling3D(pool_size=self.poolSize)(layer2)
                self.levels.append([layer1, layer2, currLayer])
            else:
                currLayer = layer2
                self.levels.append([layer1, layer2])                

        for d in reversed(range(self.depth-1)):
            upConv = self._createUpConvBlock(nFilters=currLayer._keras_shape[-1], poolSize=self.poolSize, deconvolution=self.deconvolution)(currLayer)
            concat = concatenate([upConv, self.levels[d][1]], axis=1)
            currFilters = self.levels[d][1]._keras_shape[1]
            layer1 = self._createConvBlock(inputLayer=concat, nFilters=currFilters, batchNorm=self.batchNormalization)
            currLayer = self._createConvBlock(inputLayer=layer1, nFilters=currFilters, batchNorm=self.batchNormalization)
        
        finalLayer = Conv3D(self.nLabels, (1,1,1), activation=self.activation)(currLayer)
        self.model = Model(inputs=inputs, outputs=finalLayer)
        self.model.compile(optimizer=self.optimizer(lr=self.learningRate), loss=self.lossFunction, metrics=self.metrics)
        
        return self.model

In [0]:
def stepDecay(epoch, initLR, drop, epochDrop):
    return initLR*drop**np.floor((1+epoch)/float(epochDrop))

def getCallbacks(modelFile, initLR=1e-4, lrDrop=0.5, lrEpochs=None, lrPatience=50, loggingFile='training.log', verbosity=1, earlystopPatience=None):
    callbacks = list()
    callbacks.append(ModelCheckpoint(modelFile, save_best_only=True))
    callbacks.append(CSVLogger(loggingFile, append=True))
    
    if lrEpochs:
        callbacks.append(LearningRateScheduler(partial(stepDecay, initLR=initLR, drop=lrDrop, epochDrop=lrEpochs)))
    else:
        callbacks.append(ReduceLROnPlateau(factor=lrDrop, patience=lrPatience, verbose=verbosity))
    
    if earlystopPatience:
        callbacks.append(EarlyStopping(verbose=verbosity, patience=earlystopPatience))
    
    return callbacks

First of all, the entire preprocessed dataset is stored in the `h5py` format. We shall write a custom Keras `Sequence` class to be used as a `Generator`

In [0]:
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, baseDir, listIDs, batchSize=1, dim=(160,160,16), nChannels=4, nClasses=3, shuffle=True):
    
        'Initialization'
        self.baseDir = baseDir
        self.dim = dim
        self.batchSize = batchSize
        self.listIDs = listIDs
        self.nChannels = nChannels
        self.nClasses = nClasses
        self.shuffle = shuffle
        self.indexes = None
        self.on_epoch_end()
        self.allX = {}
        self.ally = {}
        self.initData()
        
    def initData(self):
        for ids in self.listIDs:
            with h5py.File(self.baseDir+ids,'r') as f:
                self.allX[ids] = np.array(f.get('x'))
                self.ally[ids] = np.moveaxis(np.array(f.get('y')), 3, 0)[1:]
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.listIDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, tempListIDs):
        'Generates data containing batch_size samples' # X : (n_samples, n_channels, *dim)
        
        # Initialization
        X = np.zeros((self.batchSize, self.nChannels, *self.dim), dtype = np.float64)
        y = np.zeros((self.batchSize, self.nClasses, *self.dim), dtype=np.float64)
        
        # Data Generation
        for i, ids in enumerate(tempListIDs):
            #with h5py.File(self.baseDir+ids, 'r') as f:
            #    X[i] = np.array(f.get('x'))
            #    y[i] = np.moveaxis(np.array(f.get('y')), 3, 0)[1:]
            X[i] = self.allX[ids]
            y[i] = self.ally[ids]
        return X, y
    
    def __len__(self):
        'Number of batches per epoch'
        return int(np.floor(len(self.listIDs)/self.batchSize))
    
    def __getitem__(self, index):
        'generate a batch of data'
        indexes = self.indexes[index*self.batchSize: (index+1)*self.batchSize]
        sampleList = [self.listIDs[i] for i in indexes]
        X,y = self.__data_generation(sampleList)
        return X,y

In [0]:
import json
with open(DataDir+'config.json') as file:
    config = json.load(file)

In [9]:
trainGen = DataGenerator(baseDir=trainDir, listIDs=config['train'], batchSize=3, dim=(160,160,16))
validGen = DataGenerator(baseDir=validDir, listIDs=config['valid'], batchSize=3, dim=(160,160,16))
model = UNet3D(optimizer=Adam, lossFunction=softDiceLoss, metrics=[diceCoefficient]).createModel()
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 160, 160, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 160, 160, 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 160, 160, 55360       activation_1[0][0]               
___________

In [0]:
def trainModel(model, modelFile, trainGen, validGen, stepsPerEpoch, validSteps, initLR=1e-3, 
               lrDrop=0.5, lrEpochs=None, nEpochs=500, lrPatience=20, earlyStopPatience=None):
    
    callbacks = getCallbacks(modelFile=modelFile, initLR=initLR, lrDrop=lrDrop,
                            lrEpochs=lrEpochs, lrPatience=lrPatience, earlystopPatience=earlyStopPatience)
    if onColab:
        mp = True
        wrks = 16
    else:
        mp = False
        wrks = 1
    model.fit_generator(generator=trainGen,
                        steps_per_epoch=stepsPerEpoch,
                        epochs=nEpochs,
                        validation_data=validGen,
                        validation_steps=validSteps,
                        callbacks=callbacks,
                        use_multiprocessing=mp,
                        workers=wrks)

In [11]:
steps_per_epoch = 20
n_epochs=10
validation_steps = 20

model.fit_generator(generator=trainGen,
        steps_per_epoch=steps_per_epoch,
        epochs=n_epochs,
        use_multiprocessing=True,
        validation_data=validGen,
        validation_steps=validation_steps)
model.save_weights('my_model_pretrained.hdf5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 2 could not be retrieved. It could be because a worker has died.
  UserWarning)


 1/20 [>.............................] - ETA: 13:38 - loss: 0.8880 - diceCoefficient: 0.0915

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 0 could not be retrieved. It could be because a worker has died.
  UserWarning)


 2/20 [==>...........................] - ETA: 11:17 - loss: 0.8635 - diceCoefficient: 0.1103

...

 4/20 [=====>........................] - ETA: 9:18 - loss: 0.8515 - diceCoefficient: 0.1202 

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 4 could not be retrieved. It could be because a worker has died.
  UserWarning)


19/20 [===========================>..] - ETA: 10s - loss: 0.7976 - diceCoefficient: 0.1637

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 5 could not be retrieved. It could be because a worker has died.
  UserWarning)


20/20 [==============================] - 392s 20s/step - loss: 0.7928 - diceCoefficient: 0.1679 - val_loss: 0.8259 - val_diceCoefficient: 0.1669
Epoch 2/10
20/20 [==============================] - 50s 3s/step - loss: 0.5292 - diceCoefficient: 0.3788 - val_loss: 0.7171 - val_diceCoefficient: 0.2657
Epoch 3/10
20/20 [==============================] - 50s 3s/step - loss: 0.3625 - diceCoefficient: 0.5300 - val_loss: 0.7564 - val_diceCoefficient: 0.2372
Epoch 4/10
20/20 [==============================] - 50s 3s/step - loss: 0.3014 - diceCoefficient: 0.6037 - val_loss: 0.6666 - val_diceCoefficient: 0.3013
Epoch 5/10
20/20 [==============================] - 50s 3s/step - loss: 0.2471 - diceCoefficient: 0.6724 - val_loss: 0.6508 - val_diceCoefficient: 0.3222
Epoch 6/10
20/20 [==============================] - 50s 3s/step - loss: 0.1997 - diceCoefficient: 0.7441 - val_loss: 0.7106 - val_diceCoefficient: 0.2687
Epoch 7/10
20/20 [==============================] - 50s 3s/step - loss: 0.1719 - dice